In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the datetime module from the datetime library.
from datetime import datetime
# Import the API key.
from config import weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

758

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=4c3237a5d3810ea714b644930e53e5a7


In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
         
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        current_desc = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": current_desc})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")

print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | atuona
Processing Record 2 of Set 1 | turbat
Processing Record 3 of Set 1 | inta
Processing Record 4 of Set 1 | hobart
Processing Record 5 of Set 1 | saint george
Processing Record 6 of Set 1 | vaini
Processing Record 7 of Set 1 | high rock
Processing Record 8 of Set 1 | tigil
Processing Record 9 of Set 1 | saskylakh
Processing Record 10 of Set 1 | marcona
City not found. Skipping...
Processing Record 11 of Set 1 | rikitea
Processing Record 12 of Set 1 | norman wells
Processing Record 13 of Set 1 | qaanaaq
Processing Record 14 of Set 1 | albany
Processing Record 15 of Set 1 | kapaa
Processing Record 16 of Set 1 | lastoursville
Processing Record 17 of Set 1 | butaritari
Processing Record 18 of Set 1 | hilo
Processing Record 19 of Set 1 | ushuaia
Processing Record 20 of Set 1 | cape town
Processing Record 21 of Set 1 | iqaluit
Processing Record 22 of Set 1 | brae
Processing Record 23 of Set 1 | dik

In [7]:
#Checking to see if dataframe is correct.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Atuona,PF,-9.8000,-139.0333,79.75,75,94,15.37,light rain
1,Turbat,PK,26.0023,63.0440,99.68,18,67,9.42,broken clouds
2,Inta,RU,66.0317,60.1659,31.12,88,98,6.85,overcast clouds
3,Hobart,AU,-42.8794,147.3294,57.20,67,75,5.75,light rain
4,Saint George,US,37.1041,-113.5841,53.60,50,1,10.36,clear sky
5,Vaini,TO,-21.2000,-175.2000,73.40,94,75,9.22,broken clouds
6,High Rock,BS,26.6208,-78.2833,64.40,88,20,5.75,few clouds
7,Tigil,RU,57.8000,158.6667,29.71,93,70,7.09,broken clouds
8,Saskylakh,RU,71.9167,114.0833,33.57,86,80,7.67,broken clouds
9,Rikitea,PF,-23.1203,-134.9692,76.33,73,71,9.95,light rain


In [8]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")